## Import and process the public dataset


### 1. Download and extract original files
First, let's download the IDD microdata. From http://inep.gov.br/microdados, under the section heading "Indicador da Diferença entre os Desempenhos Observado e Esperado (IDD)"

In [1]:
import pandas as pd
import os, sys,zipfile
import requests
original = 'data/original'
processed = 'data/processed'


In [2]:


URLs = ["http://download.inep.gov.br/microdados/microdados_idd_2014.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2015.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2016.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2017.zip",
        "http://download.inep.gov.br/microdados/microdados_idd_2018.zip"]
for url in URLs:
    fn = url.split('/')[-1]
    if os.path.isfile(os.path.join(original,fn)):
        print("skipping, {} already download".format(fn))
    else:
        r = requests.get(url, allow_redirects=True)
        open(os.path.join(original,fn), 'wb').write(r.content)
    ano = fn.split('_')[-1][:4]
    if not os.path.isdir(os.path.join(original,ano)):
        zipf = zipfile.ZipFile(os.path.join(original,fn), 'r')
        zipf.extractall(os.path.join(original,ano))
        zipf.close()

skipping, microdados_idd_2014.zip already download
skipping, microdados_idd_2015.zip already download
skipping, microdados_idd_2016.zip already download
skipping, microdados_idd_2017.zip already download
skipping, microdados_idd_2018.zip already download


Second, the oficial results for the indicators per course, from [here](http://portal.inep.gov.br/web/guest/indicadores-de-qualidade). Ideally, we'd use the oficial spreadsheets with [the course IDDs](http://portal.inep.gov.br/web/guest/educacao-superior/indicadores-de-qualidade/indicador-de-diferenca-entre-os-desempenhos-observado-e-esperado-idd), but these data are currently only available for 2016 and 20017. Therefore, for 2014 and 2015 we use spreadsheets for the "CPC" (Conceito Preliminar do Curso) indicator, from [here](http://portal.inep.gov.br/web/guest/conceito-preliminar-de-curso-cpc-). 

However, it must be noted that before 2015, these spreadsheets only contain the IDD results aggregated by the so-called "Unidade de observação" (observed unit), which are all the courses within an institution (university or college) within one municipality. Some courses have more than one "version" within one institution and municipality (for example night-time and day-time courses), and the CPC and its inputs like the IDD were not reported separately before 2015. 

In [3]:
cpcURLs = ["http://download.inep.gov.br/educacao_superior/enade/planilhas/2012/cpc_2012_site_2014_03_14.xls",
           "http://download.inep.gov.br/educacao_superior/enade/planilhas/2013/cpc2013_atualizado_em_27112017.xls",
           "http://download.inep.gov.br/educacao_superior/enade/planilhas/2014/cpc2014_atualizado_em_04122017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/cpc_2015_portal_atualizado_03_10_2017.xls",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/Resultado_CPC_2016_portal_23_02_2018.xls",
           "http://download.inep.gov.br/educacao_superior/igc_cpc/2018/resultado_cpc_2017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2019/resultados_conceito_enade_2018.xlsx",
           "http://download.inep.gov.br/educacao_superior/igc_cpc/2018/portal_CPC_edicao2018.xlsx"
        ]

iddURLs = ["http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2017/resultado_IDD2016_portal_06_09_2017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2018/resultados_IDD_2017.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2019/resultados_IDD_2018.xlsx",
           "http://download.inep.gov.br/educacao_superior/indicadores/legislacao/2019/resultados_IDD_2018.xlsx"]

for url in cpcURLs + iddURLs:
    fn = url.split('/')[-1]
    if os.path.isfile(os.path.join(original,fn)):
        print("skipping, {} already download".format(fn))
    else:
        r = requests.get(url, allow_redirects=True)
        open(os.path.join(original,fn), 'wb').write(r.content)

skipping, cpc_2012_site_2014_03_14.xls already download
skipping, cpc2013_atualizado_em_27112017.xls already download
skipping, cpc2014_atualizado_em_04122017.xlsx already download
skipping, cpc_2015_portal_atualizado_03_10_2017.xls already download
skipping, Resultado_CPC_2016_portal_23_02_2018.xls already download
skipping, resultado_cpc_2017.xlsx already download
skipping, resultados_conceito_enade_2018.xlsx already download
skipping, portal_CPC_edicao2018.xlsx already download
skipping, resultado_IDD2016_portal_06_09_2017.xlsx already download
skipping, resultados_IDD_2017.xlsx already download
skipping, resultados_IDD_2018.xlsx already download
skipping, resultados_IDD_2018.xlsx already download


### 2. Process the orginal data
We'll put processed, intermediate data files into data/processed as csv files.

First, the **microdata**. The download and unzip above should have created the following comma separated microdata, which we will transform into files with standardized column names. For now, we'll only use the microdata of 2014 and 2017 (which are the only years for which the same areas were evaluated with the new "ENEM" methodology).

In [4]:
files = {2014:'data/original/2014/Dados/microdados_idd_2014.csv',
         2015:'data/original/2015/microdados_idd_2015/2.Dados/microdados_idd_2015.csv',
         2016:'data/original/2016/2.Dados/microdados_idd_2016.csv',
         2017:'data/original/2017/3.DADOS/MICRODADOS_IDD_2017.txt',
         2018:'data/original/2018/3.DADOS/microdados_idd_2018.txt'}

catadmmap17 = {1:'pub',2:'pub',3:'pub',4:'priv_cfl',5:'priv_sfl'}
catadmmap14 = {93:'pub',116:'pub',118:'priv_cfl',121:'priv_sfl',10001:'pub',
               10002:'pub',10003:'pub',10004:'priv_cfl',10005:'priv_cfl',
               10006:'priv_cfl',10007:'priv_sfl',10008:'priv_sfl',10009:'priv_sfl'}
def processes_idd_microdata(df):
    df = df[['ano','area','ies','catadm','orgacad','curso','municipio','ingresso','nota','cn','ch','lc','mt']]
    ano = df['ano'][0]
    if ano == 2017 | ano == 2018:
        df['adm'] = df['catadm'].map(catadmmap17)
    if ano == 2014 | ano == 2015:
        df['adm'] = df['catadm'].map(catadmmap14)
    df.to_csv(os.path.join(processed,"idd-microdata-{}.csv".format(ano)),index=False)
    return df

# 2014
oldcols = ['nu_ano','co_grupo','co_ies','co_catad','co_orgac','co_munic_curso','co_curso','ano_in_grad','tp_inscricao','tp_pres','nt_ger','enem_nt_cn','enem_nt_ch','enem_nt_lc','enem_nt_mt']
df = pd.read_table(files[2014],sep=';',usecols=oldcols)
df.columns = ['ano','area','ies','catadm','orgacad','municipio','curso','ingresso','tp_inscricao','tp_pres','nota','cn','ch','lc','mt']
df = df[df['tp_pres'] == 555] # presente com resultado válido
df = df[df['tp_inscricao'] == 0] # Concluinte
df = processes_idd_microdata(df)

# 2015
oldcols = ['nu_ano','co_grupo','co_ies','co_categad','co_orgacad','co_curso','co_munic','tp_inscricao','ano_in_grad','tp_pres','nt_ger','enem_nt_cn','enem_nt_ch','enem_nt_lc','enem_nt_mt']
df = pd.read_table(files[2015],sep=';',usecols=oldcols)
df.columns = ['ano','area','ies','catadm','orgacad','curso','municipio','tp_inscricao','ingresso','tp_pres','nota','cn','ch','lc','mt']
df = df[df['tp_pres'] == 555] # presente com resultado válido
df = df[df['tp_inscricao'] == 0] # Concluinte
df = processes_idd_microdata(df)

#2017
oldcols = ['NU_ANO','CO_IES','CO_CATEGAD','CO_ORGACAD','CO_GRUPO','CO_CURSO','CO_MUNIC_CURSO','ANO_IN_GRAD','NT_GER','VL_NOTA_ENEM_CN','VL_NOTA_ENEM_CH','VL_NOTA_ENEM_LC','VL_NOTA_ENEM_MT']
df = pd.read_table(files[2017],sep=';',usecols=oldcols)
df.columns = ['ano','ies','catadm','orgacad','area','curso','municipio','ingresso','nota','cn','ch','lc','mt']
df = processes_idd_microdata(df)

#2018
oldcols = ['NU_ANO','CO_IES','CO_CATEGAD','CO_ORGACAD','CO_GRUPO','CO_CURSO','CO_MUNIC_CURSO','ANO_IN_GRAD','NT_GER','ENEM_NT_CN','ENEM_NT_CH','ENEM_NT_LC','ENEM_NT_MT']
df = pd.read_table(files[2018],sep=';',usecols=oldcols)
df.columns = ['ano','ies','catadm','orgacad','area','curso','municipio','ingresso','nota','cn','ch','lc','mt']
df = processes_idd_microdata(df)

<ipython-input-4-5c6f35bca924>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adm'] = df['catadm'].map(catadmmap14)


Second, the **spreadsheets** with the results INEP published. Let's transform the ridiculous spreadsheets into nice csv files. We have the IDD spreadsheets for 2016 thru 2018:

In [5]:
idd18 = pd.read_excel('data/original/resultados_IDD_2018.xlsx',usecols=[1,7,8,9,15,17,19])
idd18.columns = ['area','ca','curso','modalidade','N','idd','idd_faixa']
idd18.curso = idd18.curso.astype('str')
idd18['ano'] = 2018
idd18.to_csv(os.path.join(processed,"idd-pub-2018.csv"),index=False)

In [6]:
idd17 = pd.read_excel('data/original/resultados_IDD_2017.xlsx',usecols=[1,8,15,17])
idd17.columns = ['area','curso','N','idd']
idd17.curso = idd17.curso.astype('str')
idd17['ano'] = 2017
idd17.to_csv(os.path.join(processed,"idd-pub-2017.csv"),index=False)

In [7]:
idd16 = pd.read_excel('data/original/resultado_IDD2016_portal_06_09_2017.xlsx',usecols=[6,7,15,17])
idd16.columns = ['curso','area','N','idd']
idd16 = idd16[['area','curso','N','idd']]
idd16.curso = idd16.curso.astype('str')
idd16['ano'] = 2016
idd16.to_csv(os.path.join(processed,"idd-pub-2016.csv"),index=False)

But the same information is in the **CPC spreadsheets**. Unfortunately, as noted above, in the CPC spreadsheets the "código do curso" is only identified after 2015: "O CPC, assim como o Conceito Enade, também passou a ser calculado, a partir da edição de 2015, por curso de graduação, identificado pelo código do curso constante no Sistema e-MEC" (see [here](http://portal.inep.gov.br/conceito-preliminar-de-curso-cpc-))

In [14]:
cpccatadmmap17 = {'Privada com fins lucrativos':'priv_cfl',
                  'Privada sem fins lucrativos':'priv_sfl',
                  'Pública Estadual':'pub',
                  'Pública Federal':'pub',
                  'Pública Municipal':'pub'
                 }

cpccatadmmap15 = {
                  'Federal': 'pub',
                  'Estadual': 'pub',
                  'Municipal': 'pub',
                  'Privada': 'priv'
}

cpccatadmmap14 = {"Pessoa Jurídica de Direito Público - Federal":'pub',
                  'Pessoa Jurídica de Direito Público - Municipal':'pub',
                  'Pessoa Jurídica de Direito Público - Estadual':'pub',
                  'Privada sem fins lucrativos':'priv_sfl',
                  'Privada com fins lucrativos':'priv_cfl',
                  'Pessoa Jurídica de Direito Privado - Sem fins lucrativos - Fundação':'priv_sfl',
                  'Pessoa Jurídica de Direito Privado - Com fins lucrativos - Associação de Utilidade Pública': 'priv_cfl',
                  'Pessoa Jurídica de Direito Privado - Sem fins lucrativos - Associação de Utilidade Pública':'priv_sfl',
                  'Pessoa Jurídica de Direito Privado - Com fins lucrativos - Sociedade Mercantil ou Comercial':'priv_cfl',
                  'Pessoa Jurídica de Direito Privado - Com fins lucrativos - Sociedade Civil':'priv_cfl',
                  'Pessoa Jurídica de Direito Privado - Sem fins lucrativos - Fundação':'priv_sfl',
                  'Pessoa Jurídica de Direito Privado - Sem fins lucrativos - Sociedade':'priv_sfl',
                 }

def xls2csv(ano,fn,cols,colnames):
    df = pd.read_excel(fn,usecols=cols,na_values=['-','.'])
    df.columns = colnames
    #if ano in [2015,2016]:
    #    df = df[['area','ies','municipio','curso','N','fracN','idd','cpc','cpcf']]
    #if ano == 2012:
    #    df = df[['area','ies','municipio','Ncursos','N','idd','cpc','cpcf']]
    df['ano'] = ano
    if ano == 2017 | ano == 2018:
        df['adm'] = df['catadm'].map(cpccatadmmap17)
    if ano == 2015:
        print(df.columns)
        df['adm'] = df['catadm'].map(cpccatadmmap15)
    if ano == 2014:
        df['adm'] = df['catadm'].map(cpccatadmmap14)
    
    return df

In [15]:
cpcfiles = {2018: 'data/original/portal_CPC_edicao2018.xlsx',
            2017: 'data/original/resultado_cpc_2017.xlsx',
            2016: 'data/original/Resultado_CPC_2016_portal_23_02_2018.xls',
            2015: 'data/original/cpc_2015_portal_atualizado_03_10_2017.xls',
            2014: 'data/original/cpc2014_atualizado_em_04122017.xlsx',
            2013: 'data/original/cpc2013_atualizado_em_27112017.xls',
            2012: 'data/original/cpc_2012_site_2014_03_14.xls'}

# catadm = federal, privado, etc. Mapeado em pub, priv_sfl (sem fins lucrativos) e priv_cfl
# modalidade = EaD ou presencial
# Nci, Ncp e N = participantes inscritos, participantes e com ENEM resp.


cpccols = {
           2018: [[1,3,4,5,6,7,9,10,13,14,19,20,21,22,36,37],['ies','siglies','orgacad','catadm','curso','area','modalidade','municipio','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']],
           2017: [[1,3,5,6,7,8,9,10,13,14,17,18,19,20,35,36],['area','ies','siglies','orgacad','catadm','curso','modalidade','municipio','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']], #N = conluintes com nota no enem
           2016: [[1,6,7,10,18,19,20,35,36],['ies','curso','area','municipio','N','fracN','idd','cpc','cpcf']], #idd = Nota Bruta - IDD
           2015: [[1,4,5,6,7,10,13,14,17,18,19,20,35,36], ['ies','orgacad','catadm','curso','area','municipio','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']],
           2014: [[1,3,5,6,7,8,12,13,14,18,25,26,27,37,38], ['area','ies','siglies','catadm','orgacad','municipio','Ncursos','Nci','Ncp','enade','N','fracN','idd','cpc','cpcf']],
           2013: [[1,3,8,12,14,29,37,38], ['area','ies','municipio','Ncursos','N','idd','cpc','cpcf']],
           2012: [[1,3,8,9,13,24,36,37], ['area','ies','Ncursos','municipio','N','idd','cpc','cpcf']]}

for ano in range(2012,2019):
    #print(ano)
    df = xls2csv(ano,cpcfiles[ano],cols=cpccols[ano][0],colnames = cpccols[ano][1])
    df.to_csv(os.path.join(processed,"cpc-pub-{}.csv".format(ano)),index=False)

2012
2013
2014
2015
Index(['ies', 'orgacad', 'catadm', 'curso', 'area', 'municipio', 'Nci', 'Ncp',
       'enade', 'N', 'fracN', 'idd', 'cpc', 'cpcf', 'ano'],
      dtype='object')
2016
2017
2018


In [11]:
ano = 2015
df = pd.read_excel(cpcfiles[ano],usecols=cpccols[ano][0],na_values=['-','.'])
df.sample(5)

,Código da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,Código da Área,Código do Município,Concluintes Inscritos,Concluintes Participantes,Nota Contínua do Enade,Concluintes Participantes com nota no Enem,Percentual de Concluintes participantes com nota no Enem,Nota Bruta - IDD,CPC Contínuo,CPC Faixa
5214,515,Centro Universitário,Privada,1158619,84,3303302,28,19,1.918355,5.0,0.263158,0.609288,2.854730,3
7595,489,Centro Universitário,Privada,104324,803,3306305,37,34,2.127999,25.0,0.735294,-0.398648,2.896846,3
7023,11,Universidade,Privada,52551,101,2611606,39,28,3.190678,3.0,0.107143,NaN,3.468615,4
3638,195,Faculdade,Privada,5380,22,3304557,219,182,3.050960,61.0,0.335165,-0.216512,2.655309,3
1176,1843,Faculdade,Privada,1048101,1,3111200,36,36,1.003881,15.0,0.416667,-3.141277,NaN,Curso não reconhecido até 31/12/2015
